In [1]:
import sqlite3
import ipywidgets as widgets
from IPython.display import display
import datetime
from sqlite3 import Error

# Clase Database para manejar las operaciones con SQLite
class Database:
    def __init__(self, db_name):
        """Inicializa la conexión a la base de datos."""
        self.db_name = db_name
        self.conn = None
        self.cursor = None
        try:
            self.conn = sqlite3.connect(self.db_name)
            self.cursor = self.conn.cursor()
            print(f"Conexión establecida con la base de datos {self.db_name}")
        except Error as e:
            print(f"Error al conectar a la base de datos: {e}")

    def tabla_existe(self, table_name):
        """Verifica si una tabla existe en la base de datos."""
        try:
            self.cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            result = self.cursor.fetchone()
            return bool(result)
        except Error as e:
            print(f"Error al verificar la existencia de la tabla: {e}")
            return False

    def create_table(self, table_name, create_table_sql):
        """Crea una nueva tabla en la base de datos, si no existe."""
        if not self.tabla_existe(table_name):
            try:
                create_table_sql = create_table_sql.format(table_name)
                self.cursor.execute(create_table_sql)
                self.conn.commit()
                print(f"Tabla {table_name} creada correctamente.")
            except Error as e:
                print(f"Error al crear la tabla: {e}")

    def drop_table(self, table_name):
        """Elimina una tabla de la base de datos, si existe."""
        if self.tabla_existe(table_name):
            try:
                self.cursor.execute(f"DROP TABLE {table_name}")
                self.conn.commit()
                print(f"Tabla {table_name} eliminada correctamente.")
            except Error as e:
                print(f"Error al eliminar la tabla: {e}")
        else:
            print(f"La tabla {table_name} no existe y no se puede eliminar.")
            
    def get_tablas(self):
        """Obtiene las tablas disponibles en la base de datos."""
        try:
            self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = self.cursor.fetchall()
            return [table[0] for table in tables]
        except Error as e:
            print(f"Error al obtener las tablas: {e}")
            return []

    def insert(self, table, columns, values):
        """Inserta un nuevo registro en una tabla."""
        try:
            placeholders = ", ".join("?" * len(values))
            sql = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({placeholders})"
            self.cursor.execute(sql, values)
            self.conn.commit()
            print("Registro insertado correctamente.")
        except Error as e:
            print(f"Error al insertar el registro: {e}")

    def update(self, table, columns, values, where):
        """Actualiza un registro existente en una tabla."""
        try:
            set_clause = ", ".join([f"{col} = ?" for col in columns])
            sql = f"UPDATE {table} SET {set_clause} WHERE {where}"
            self.cursor.execute(sql, values)
            self.conn.commit()
            print("Registro actualizado correctamente.")
        except Error as e:
            print(f"Error al actualizar el registro: {e}")

    def select(self, table, columns="*", where=None):
        """Consulta los registros de una tabla."""
        try:
            sql = f"SELECT {columns} FROM {table}"
            if where:
                sql += f" WHERE {where}"
            self.cursor.execute(sql)
            return self.cursor.fetchall()
        except Error as e:
            print(f"Error al consultar los registros: {e}")
            return None

    def close(self):
        """Cierra la conexión a la base de datos."""
        if self.conn:
            self.conn.close()
            print("Conexión cerrada.")

# Función para obtener la fecha actual
def obtener_fecha():
    """Obtiene la fecha y hora actual"""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Conexión a la base de datos 'inventario.db'
db_inventario = Database('inventario.db')

# Obtener las tablas de la base de datos
tablas_inventario = db_inventario.get_tablas()

# Función para actualizar los items en el widget Select
def actualizar_items(change):
    tabla_seleccionada = tabla_dropdown.value
    if tabla_seleccionada:
        registros = db_inventario.select(tabla_seleccionada, "item")
        items = [registro[0] for registro in registros] if registros else []
        item_select.options = items
    else:
        item_select.options = []

# Función para guardar los datos en la tabla seleccionada
def guardar_datos(b):
    tabla_seleccionada = tabla_dropdown.value
    item = item_text.value
    stock = stock_number.value
    costo = costo_number.value
    usuario = usuario_text.value
    fecha = obtener_fecha()  # Fecha actual de modificación
    
    # Asegurarse de que todos los campos estén completos
    if not all([item, stock, costo, usuario]):
        with mensaje_output:
            mensaje_output.clear_output()
            print("Por favor, complete todos los campos antes de guardar.")
        return

    # Insertar los datos en la base de datos
    columns = ['item', 'stock', 'costo', 'usuario', 'fecha']
    values = (item, stock, costo, usuario, fecha)
    db_inventario.insert(tabla_seleccionada, columns, values)
    
    # Limpiar los campos después de guardar
    item_text.value = ''
    stock_number.value = 0
    costo_number.value = 0.0
    usuario_text.value = ''

    # Mostrar mensaje de éxito
    with mensaje_output:
        mensaje_output.clear_output()
        print("Datos guardados correctamente!")

# Función para visualizar los datos de la tabla seleccionada
def ver_datos(b):
    with output_table:
        output_table.clear_output()
        tabla_seleccionada = tabla_dropdown.value
        registros = db_inventario.select(tabla_seleccionada)
        if registros:
            for registro in registros:
                print(registro)

# Función para crear una nueva tabla
def crear_tabla(b):
    tabla_nombre = nueva_tabla_text.value
    contraseña = contraseña_text.value

    # Verificar contraseña
    if contraseña != "12345":
        with mensaje_output:
            mensaje_output.clear_output()
            print("Contraseña incorrecta.")
        return

    # Validar nombre de la tabla
    if not tabla_nombre:
        with mensaje_output:
            mensaje_output.clear_output()
            print("Por favor, ingrese un nombre para la nueva tabla.")
        return

    # SQL para crear la nueva tabla
    create_table_sql = f'''
    CREATE TABLE IF NOT EXISTS {tabla_nombre} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        item TEXT NOT NULL,
        stock INT NOT NULL,
        costo FLOAT NOT NULL,
        usuario TEXT NOT NULL,
        fecha TEXT NOT NULL
    );
    '''
    
    db_inventario.create_table(tabla_nombre, create_table_sql)
    
    # Limpiar los campos después de crear la tabla
    nueva_tabla_text.value = ''
    contraseña_text.value = ''

    # Actualizar la lista de tablas disponibles
    tablas_inventario = db_inventario.get_tablas()
    tabla_dropdown.options = tablas_inventario
    tabla_dropdown.value = tablas_inventario[-1] if tablas_inventario else None

    with mensaje_output:
        mensaje_output.clear_output()
        print(f"Tabla '{tabla_nombre}' creada correctamente.")

# Función para actualizar el item seleccionado en la base de datos
def actualizar_item_en_base_de_datos(b):
    tabla_seleccionada = tabla_dropdown.value
    item_seleccionado = item_select.value
    nuevo_nombre = nombre_item_text.value
    nuevo_stock = stock_update_number.value
    nuevo_costo = costo_update_number.value
    usuario = usuario_update_text.value
    fecha = obtener_fecha()

    # Verificar si todos los campos están completos
    if not all([nuevo_nombre, nuevo_stock, nuevo_costo, usuario]):
        with mensaje_output:
            mensaje_output.clear_output()
            print("Por favor, complete todos los campos antes de actualizar.")
        return

    # Actualizar el item en la base de datos
    columns = ['item', 'stock', 'costo', 'usuario', 'fecha']
    values = (nuevo_nombre, nuevo_stock, nuevo_costo, usuario, fecha)
    where_clause = f"item = '{item_seleccionado}'"  # Usar el item seleccionado como filtro
    db_inventario.update(tabla_seleccionada, columns, values, where_clause)

    # Limpiar los campos después de actualizar
    nombre_item_text.value = ''
    stock_update_number.value = 0
    costo_update_number.value = 0.0
    usuario_update_text.value = ''
    item_select.value = None

    # Mostrar mensaje de éxito
    with mensaje_output:
        mensaje_output.clear_output()
        print("Item actualizado correctamente!")

# Definición de widgets

# Dropdown para seleccionar la tabla de inventario
tabla_dropdown = widgets.Dropdown(
    options=tablas_inventario,
    value=tablas_inventario[0] if tablas_inventario else None,
    description='Tabla:',
    disabled=False
)

# Entrada de texto para el nombre del item
item_text = widgets.Text(
    value='',
    placeholder='Escriba el nombre del item',
    description='Item:',
    disabled=False
)

# Entrada de número entero para stock
stock_number = widgets.IntText(
    value=0,
    description='Stock:',
    disabled=False
)

# Entrada flotante para costo
costo_number = widgets.FloatText(
    value=0.0,
    description='Costo:',
    disabled=False
)

# Entrada de texto para el usuario
usuario_text = widgets.Text(
    value='',
    placeholder='Ingrese el nombre del usuario',
    description='Usuario:',
    disabled=False
)

# Botón para guardar los datos
guardar_button = widgets.Button(description="Guardar Data")
guardar_button.on_click(guardar_datos)

# Botón para visualizar los datos de la tabla seleccionada
ver_datos_button = widgets.Button(description="Ver Datos")
ver_datos_button.on_click(ver_datos)

# Crear el widget Output para mostrar los registros
output_table = widgets.Output()

# Crear el widget Output para mostrar mensajes
mensaje_output = widgets.Output()

# Botón para actualizar el item
update_button = widgets.Button(description="Actualizar")
update_button.on_click(actualizar_item_en_base_de_datos)

# Entrada de texto para crear nueva tabla
nueva_tabla_text = widgets.Text(
    value='',
    placeholder='Nombre de la tabla',
    description='Nueva Tabla:',
    disabled=False
)

# Entrada de texto para la contraseña de seguridad
contraseña_text = widgets.Password(
    value='',
    placeholder='Contraseña',
    description='Contraseña:',
    disabled=False
)

# Botón para crear nueva tabla
crear_tabla_button = widgets.Button(description="Crear Tabla")
crear_tabla_button.on_click(crear_tabla)

# Selector de item para actualizar
item_select = widgets.Select(
    description='Item:',
    disabled=False,
)

# Entrada de texto para el nuevo nombre del item
nombre_item_text = widgets.Text(
    value='',
    placeholder='Nuevo nombre del item',
    description='Nuevo Item:',
    disabled=False
)

# Entrada de número para el nuevo stock
stock_update_number = widgets.IntText(
    value=0,
    description='Nuevo Stock:',
    disabled=False
)

# Entrada de número flotante para el nuevo costo
costo_update_number = widgets.FloatText(
    value=0.0,
    description='Nuevo Costo:',
    disabled=False
)

# Entrada de texto para el usuario al actualizar
usuario_update_text = widgets.Text(
    value='',
    placeholder='Usuario que actualiza',
    description='Usuario:',
    disabled=False
)

# Crear la estructura de widgets para la primera columna
columna_1 = widgets.VBox([
    widgets.Label("INGRESAR REGISTROS A LA BD"),
    tabla_dropdown, 
    item_text, 
    stock_number, 
    costo_number, 
    usuario_text, 
    guardar_button,
    widgets.Label("GENERAR TABLA DE REGISTRO"),
    nueva_tabla_text, 
    contraseña_text,
    crear_tabla_button,
    widgets.Label("ACTUALIZAR ITEM"),
    item_select,
    nombre_item_text,
    stock_update_number,
    costo_update_number,
    usuario_update_text,
    update_button,
    mensaje_output
])

# Crear la estructura de widgets para la segunda columna (visualización)
columna_2 = widgets.VBox([output_table, ver_datos_button])

# Mostrar los widgets en dos columnas
display(widgets.HBox([columna_1, columna_2]))

# Asignar la función que actualiza los items al evento de selección de tabla
tabla_dropdown.observe(actualizar_items, names='value')


Conexión establecida con la base de datos inventario.db
